# Loop 11 Analysis: Strategy Review

## Key Observations:
1. Target: 68.919154 (lower is better)
2. Current best: 70.659959 (saspav baseline)
3. Gap: 1.74 points (2.5%)
4. Public LB best: 71.19 (terry_u16)

**CRITICAL INSIGHT**: Our target (68.92) is BELOW the current public LB best (71.19)!
This means we need to find techniques that go BEYOND what's publicly available.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load session state to analyze experiments
import json
with open('/home/code/session_state.json', 'r') as f:
    state = json.load(f)

print("=" * 60)
print("EXPERIMENT HISTORY")
print("=" * 60)
for exp in state['experiments']:
    print(f"{exp['id']}: {exp['name'][:40]:40s} | CV: {exp['cv_score']:.6f}")

print("\n" + "=" * 60)
print("SUBMISSIONS")
print("=" * 60)
for sub in state.get('submissions', []):
    print(f"{sub['experiment_id']}: CV={sub.get('cv_score', 'N/A'):.6f} | LB={sub.get('lb_score', 'N/A')}")

In [ ]:
# Key insight: All experiments converge to the same score (70.66)
# This is a STRONG LOCAL OPTIMUM

scores = [exp['cv_score'] for exp in state['experiments']]
print("Score distribution:")
print(f"  Min: {min(scores):.6f}")
print(f"  Max: {max(scores):.6f}")
print(f"  Range: {max(scores) - min(scores):.9f}")

# Count how many experiments are at the same score
from collections import Counter
score_counts = Counter([round(s, 4) for s in scores])
print(f"\nScore frequency:")
for score, count in sorted(score_counts.items()):
    print(f"  {score:.4f}: {count} experiments")

In [ ]:
# What approaches have been tried?
print("=" * 60)
print("APPROACHES TRIED")
print("=" * 60)

approaches = {}
for exp in state['experiments']:
    model_type = exp.get('model_type', 'unknown')
    if model_type not in approaches:
        approaches[model_type] = []
    approaches[model_type].append({
        'name': exp['name'],
        'score': exp['cv_score']
    })

for approach, exps in approaches.items():
    print(f"\n{approach.upper()}:")
    for e in exps:
        print(f"  - {e['name'][:50]:50s} | {e['score']:.6f}")

In [ ]:
# The key insight from web search and leaderboard:
print("=" * 60)
print("STRATEGIC ANALYSIS")
print("=" * 60)

print("""
CRITICAL FINDING:
- Public LB best: 71.19
- Our target: 68.92
- Our current: 70.66

The target is 2.27 points BELOW the public best!
This means:
1. Top teams have better solutions they haven't shared
2. We need to discover techniques independently
3. Simple optimization of public solutions won't work

WHAT WE'VE TRIED (all converge to 70.66):
- Pre-optimized baselines
- C++ SA optimizer
- Lattice construction + SA
- Eazy optimizer (gradient-based)
- Rotation optimization
- bbox3 optimizer (short runs)
- Tree removal technique

WHAT WE HAVEN'T TRIED:
1. LONG optimization runs (3+ hours) - bbox3-runner uses 3 hours
2. Multi-phase optimization with parameter sweeps
3. Genetic algorithms with crossover
4. Constraint programming (CP-SAT)
5. Different starting points (not just saspav)
6. Asymmetric packing strategies
""")

In [ ]:
# Analyze per-N scores to find improvement opportunities
baseline_path = '/home/code/external_data/saspav/santa-2025.csv'
df = pd.read_csv(baseline_path)

# Calculate per-N scores
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union

getcontext().prec = 30
scale_factor = Decimal("1e18")

def get_score_for_n(n, df):
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    if len(group_data) == 0:
        return None
    
    # Get bounding box
    xs = []
    ys = []
    for _, row in group_data.iterrows():
        x = float(str(row['x'])[1:])
        y = float(str(row['y'])[1:])
        xs.append(x)
        ys.append(y)
    
    # Approximate bounding box (simplified)
    side = max(max(xs) - min(xs), max(ys) - min(ys)) + 1.0  # Add tree size
    return side**2 / n

print("Per-N score analysis (simplified):")
scores_by_n = []
for n in range(1, 201):
    score = get_score_for_n(n, df)
    if score:
        theoretical = 0.355 * n  # Approximate theoretical minimum
        efficiency = theoretical / score if score > 0 else 0
        scores_by_n.append({'n': n, 'score': score, 'efficiency': efficiency})

scores_df = pd.DataFrame(scores_by_n)
print(f"\nWorst efficiency (most room for improvement):")
print(scores_df.nsmallest(10, 'efficiency')[['n', 'score', 'efficiency']])

In [ ]:
# Summary and recommendation
print("=" * 60)
print("RECOMMENDATION FOR NEXT EXPERIMENT")
print("=" * 60)

print("""
Given the constraints:
1. Target is BELOW public best - need novel techniques
2. Local optimization shows negligible improvement
3. The baseline is at an extremely strong local optimum

RECOMMENDED APPROACH:

1. RUN BBOX3 FOR 2-3 HOURS with full parameter sweep
   - Phase A: 2-min runs with n=[1000,1200,1500,1800,2000], r=[30,60,90]
   - Phase B: 10-min runs on top 6 candidates
   - Phase C: 20-min runs on top 3 candidates
   - Apply fix_direction() after each run
   - Use overlap repair with donor solution

2. If long runs don't help, try GENETIC ALGORITHM:
   - Crossover of good configurations
   - Mutation with small perturbations
   - Selection based on score

3. Focus on N=1-10 which have worst efficiency

The key insight: We need to ESCAPE the local optimum, not optimize within it.
Long optimization runs with diverse parameters may find different basins.
""")

print("\nSubmission exp_010 achieved CV=70.659958437 with 0 overlaps.")
print("This should be submitted to verify it passes Kaggle validation.")